# Naive‬‬ ‫‪Bayes‬‬ ‫‪Classifier‬‬

#### Fateme Seyyed Dabbaghi 810197529
#### In this project we are going to analyze the comments of products of an online market to guess whether the product is recommended or not based on each comment using naive bayes classifier 

to achieve our goal we have two datas; train data and test data. using train data after merging each title and comment of a specific comment we calculate the probability of each word appearing in category "recommended" and "not recommended". then for each comment of test data we calculate the total probability by Bayes rule for both categories. by comparing those two probabities we decide whether the comment belongs to first category or the second.we use hazm library for normalizing data.

In [1]:
from __future__ import unicode_literals
from hazm import *
import pandas as pd
import numpy as np
import collections

In [2]:
train_data = pd.read_csv ('comment_train.csv')
test_data = pd.read_csv ('comment_test.csv')

there are 3 steps down here for normalizing data:
<br>1) normalize: this function is used for replacing space with half-space if necessary. this function makes data integrated so that data proccesing will be easier.
<br>2) stemming: stemming is the process of producing morphological variants of a root/base word. but it may be wrong in some cases like when two words are stemmed from the same root that are of different stems. while calculating the probability of each word this method helps in a way that it turns all different words made by unique root to root so that the probabilities will be more accurate.
<br>3) ‫‪lemmatization‬‬: The goal of lemmatization same as stemming is to reduce inflectional forms and sometimes derivationally related forms of a word to a common base form.
<br>Stemming usually refers to a  process that chops off the ends of words in the hope of achieving this goal correctly most of the time. Lemmatization usually refers to doing things properly with the use of a vocabulary and morphological analysis of words, normally aiming to remove inflectional endings only and to return the base or dictionary form of a word. If confronted with the token saw, stemming might return just s, whereas lemmatization would attempt to return either see or saw.

In [3]:
recommend_train = train_data[(train_data['recommend'] == 'recommended')]
recommend_count = recommend_train.count()['title']
not_recommend_train = train_data[(train_data['recommend'] == 'not_recommended')]
not_recommend_count = not_recommend_train.count()['title']

normalizer = Normalizer()
stemmer = Stemmer()
lemmatizer = Lemmatizer()

def normalize_str(str):
    data = word_tokenize(normalizer.normalize(str))
    for i in range(len(data)):
        data[i] = lemmatizer.lemmatize(stemmer.stem(data[i]))
    
    return data

In [4]:
def calc_freq(data):
    count = dict(collections.Counter(data))
    return count    

def normalize_data(flag):    
    rec_words = []
    not_rec_words = []

    for index,row in recommend_train.iterrows():
        if flag == 1: rec_words += normalize_str(row['title'] + ' ' + row['comment'])
        else: rec_words += word_tokenize(row['title'] + ' ' + row['comment'])
        
    for index,row in not_recommend_train.iterrows():
        if flag == 1: not_rec_words += normalize_str(row['title'] + ' ' + row['comment'])
        else: not_rec_words += word_tokenize(row['title'] + ' ' + row['comment'])
        
    rec_data = calc_freq(rec_words)
    not_rec_data = calc_freq(not_rec_words)

    test = test_data.copy()

    for index,row in test.iterrows():
        if flag == 1: temp = normalize_str(row['title'] + ' ' + row['comment'])
        else: temp = word_tokenize(row['title'] + ' ' + row['comment'])
        for i in temp:
            if i not in rec_data: rec_data[i] = 0
            if i not in not_rec_data: not_rec_data[i] = 0
                
    return rec_data,not_rec_data,test

we calculate the probability of each word by counting the occurrences of it in train data. we may not have specific word that accured in comment in test data in one or both categories. as we multiply the probabilies this leads the probability of comment to zero and its just one word and we could guess the comment category by other words. or in another case when we have that word in one of the categories we say for sure that the comment belongs to that category because the other probability is zero.
<br>additive smoothing: In statistics, additive smoothing is a technique to smooth categorical data. additive smoothing is introduced to solve the problem of zero probability. for applying it we add 1 to the count of all words in the train data, so when a word has not accured in the train its probability will not be zero as its count is now 1. zero probability will not accur and cases like above will be solved.

In [5]:
def additive_smoothing(rec_data,not_rec_data):
    rec = {}
    not_rec = {}
    for i in rec_data:
        rec[i] = rec_data[i] + 1
    for i in not_rec_data:
        not_rec[i] = not_rec_data[i] + 1
    return rec, not_rec

1) ‫‪evidence: P(comment) is the likelihood. we ignore it in this project!
<br>2) likelihood: P(comment|recommended (or not recommended)) is the likelihood. we calculate it by multiplying all P(x|recommended (or not recommended))s, where x is each word used in comment.
<br>3) prior: P(recommended (or not recommended)) is the prior probability of class. we calculate it by counting all of the recommended (or not recommended) comments in test data and devide it by number of all comments.
<br>4) ‪posterior:P(recommended(or not recommended)|comment) is the posterior probability. it is the probability of being recommended or not recommended if the comment is what it is. we calculate it by multiplying likelihood and prior.

In [6]:
def sum_dict(dic):
    sum = 0
    for i in dic: 
        sum = sum + dic[i]
    return sum

def rec_or_not(test_,rec_data,not_rec_data,flag):
    rec_words_count = sum_dict(rec_data)
    not_rec_words_count = sum_dict(not_rec_data)
    for index,row in test_.iterrows():
        rec_prob = recommend_count/(recommend_count+not_recommend_count)
        not_rec_prob = not_recommend_count/(recommend_count+not_recommend_count)
        if flag == 1: temp = normalize_str(row['title'] + ' ' + row['comment'])
        else: temp = word_tokenize(row['title'] + ' ' + row['comment'])
        for i in temp:
            rec_prob *= (rec_data[i]/rec_words_count)
            not_rec_prob *= (not_rec_data[i]/not_rec_words_count)
            if rec_prob >= not_rec_prob: row['recommend'] = 'recommended'
            else: row['recommend'] = 'not_recommended'
    return test_

presicion: when we want to calculate whether our way for guessing the category is accurate or not we divide all the right "recommended" categoriy guesses by all of of the "recommended" guesses even if they are wrong. we may just find one of the comments recommended and it may be true. in this case our accuracy will be 1 and we may be 100% right but in fact we are far from the right answer.
<br><br>recall: when we want to calculate whether our way for guessing the category is accurate or not we divide all the right "recommended" categoriy guesses by all of of the "recommended" categories which are the actual answer. in this way if we found that all of the comments are in the "recommended" category the above way for finding the accuracy will be one again as we know for sure that its not true.
<br><br> for F1 we use harmonic mean. The harmonic mean helps to find multiplicative or divisor relationships between fractions without worrying about common denominators. Harmonic means are often used in averaging things like rates.

In [7]:
def accuracy(test):
    wrong = []
    right = 0
    for i in range(test.count()['title']):
        if test['recommend'][i] == test_data['recommend'][i] : right += 1
        else: wrong.append((test['title'][i],test['comment'][i],test['recommend'][i]))
    return right/test.count()['title'],wrong

def precision(test):
    right = 0
    for i in range(test.count()['title']):
        if test['recommend'][i] == test_data['recommend'][i] and test['recommend'][i] == 'recommended': right += 1
    return right/(test[(test['recommend'] == 'recommended')].count()['title'])

def recall(test):
    right = 0
    for i in range(test.count()['title']):
        if test['recommend'][i] == test_data['recommend'][i] and test['recommend'][i] == 'recommended': right += 1
    return right/(test_data[(test_data['recommend'] == 'recommended')].count()['title'])

def f1(pre,rec):
    return (2*pre*rec)/(pre+rec)

as we see in the answers below from a to d step by step the percentage of correctness of our answer decreases. the reason is step by step we are not doing the things that helps us to find the correct answer. like in step b we are not using preprocceses and it is certainly helpful as we said in the above. or in c and d we are not using additive smoothing method and as we expected, our answer will be less accurate.

In [8]:
#pre process and additive smoothing
rec_data_1,not_rec_data_1,test_1 = normalize_data(1)

both_rec, both_not_rec = additive_smoothing(rec_data_1,not_rec_data_1)

filled_test_both = rec_or_not(test_1,both_rec,both_not_rec,1)

acc_1,wrong = accuracy(filled_test_both)
prec_1 = precision(filled_test_both)*100
rec_1 = recall(filled_test_both)*100
f1_1 = f1(prec_1,rec_1)

print("pre process and additive smoothing:")
print ("accuracy : ",acc_1*100,"%")
print ("precision: ",'{:.5f}'.format(prec_1),"%")
print ("recall : ",rec_1,"%")
print ("F1 : ",'{:.5f}'.format(f1_1),"%")

pre process and additive smoothing:
accuracy :  91.75 %
precision:  89.38679 %
recall :  94.75 %
F1 :  91.99029 %


In [9]:
#additive smoothing
rec_data_2,not_rec_data_2,test_2 = normalize_data(0)
additive_rec, additive_not_rec = additive_smoothing(rec_data_2,not_rec_data_2)
filled_test_additive = rec_or_not(test_2,additive_rec,additive_not_rec,0)

acc_2,r = accuracy(filled_test_additive)
prec_2 = precision(filled_test_additive)*100
rec_2 = recall(filled_test_additive)*100
f1_2 = f1(prec_2,rec_2)

print("additive smoothing:")
print ("accuracy : ",acc_2*100,"%")
print ("precision: ",'{:.5f}'.format(prec_2),"%")
print ("recall : ",rec_2,"%")
print ("F1 : ",'{:.5f}'.format(f1_2),"%")

additive smoothing:
accuracy :  91.375 %
precision:  88.57809 %
recall :  95.0 %
F1 :  91.67672 %


In [10]:
#pre process
rec_data_3,not_rec_data_3,test_3 = normalize_data(1)

filled_test_pre = rec_or_not(test_3,rec_data_3,not_rec_data_3,1)

acc_3,r = accuracy(filled_test_pre)
prec_3 = precision(filled_test_pre)*100
rec_3 = recall(filled_test_pre)*100
f1_3 = f1(prec_3,rec_3)

print("pre process:")
print ("accuracy : ",acc_3*100,"%")
print ("precision: ",'{:.5f}'.format(prec_3),"%")
print ("recall : ",rec_3,"%")
print ("F1 : ",'{:.5f}'.format(f1_3),"%")

pre process:
accuracy :  86.0 %
precision:  79.50820 %
recall :  97.0 %
F1 :  87.38739 %


In [11]:
#none
rec_data_4,not_rec_data_4,test_4 = normalize_data(0)

filled_test_none = rec_or_not(test_4,rec_data_4,not_rec_data_4,0)

acc_4,r = accuracy(filled_test_none)
prec_4 = precision(filled_test_none)*100
rec_4 = recall(filled_test_none)*100
f1_4 = f1(prec_4,rec_4)

print("none:")
print ("accuracy : ",acc_4*100,"%")
print ("precision: ",'{:.5}'.format(prec_4),"%")
print ("recall : ",rec_4,"%")
print ("F1 : ",'{:.5f}'.format(f1_4),"%")

none:
accuracy :  85.625 %
precision:  78.443 %
recall :  98.25 %
F1 :  87.23640 %


we will discuss one of the comments which was expected to be "not recommended" but is is "recommended":
<br><br>'برد بالایی نداره کیفیت صداش معمولیه برای تایم تمرین و ورزش مناسبه از روی گوش نمیفته  ظریف نیست میان نمونه های هم رده خودش خیلی خوب نیست'
<br><br>as we can see in this comment good attributes are used with negetive verb. as verbs are less than the attributes they are less likely to make an impact. we could use weighted probability which verbs weight could be more.

In [12]:
df = pd.DataFrame(wrong, columns =['title', 'comment', 'recommend']) 
df

,title,comment,recommend
0,وری گود,تازه خریدم یه مدت کار بکنه مشخص میشه کیفیت قطعاتش,not_recommended
1,دستگاه خیلی ضعیف,من این فیس براس چند روز یپش به دستم رسید و الا...,recommended
2,خوب ولی کارایی محدود,مدل 46MM به دست شما نخواهد رسید و به جای آن مد...,recommended
3,نقد پس از خرید,سلام ، راحت شدم از کابل شارژ ، توصیه میشود به ...,not_recommended
4,نقد منصفانه,من تو تخفیف ویژه 5 تا خریدم و همشون رو هم تست ...,recommended
...,...,...,...
61,عدم بسته بندی و تحویل مناسب,خیلی نردبان خوبیه خیلی بدرد بخوره تنها نکته من...,not_recommended
62,ظاهر خراب,شکل و ظاهر محصول که خیلی خط و خش داشت و پایینش...,recommended
63,پیشنهاد نمیدم,من دوسه ماهی هست این کفشدازردیجی گرفتم متاسفان...,not_recommended
64,SanDisk,سلام وقتتون بخیر\r\nاولش که من این USB رو خرید...,recommended
